In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import numpy
import string
import random
from sklearn import linear_model
import numpy as np

In [4]:
import homework3

In [5]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [6]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [7]:
def countRight(a,b,epsilon):
    if len(a) != len(b):
        print("It looks like your solution has the wrong length (got " + str(len(a)) + ", expected "
 + str(len(b)) + ")")
        return 0
    a_ = np.array(a).flatten()
    b_ = np.array(b).flatten()
    right = np.abs(a_ - b_) < epsilon
    return float(sum(right) / len(right))

In [8]:
# Some data structures that will be useful

In [9]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [10]:
len(allRatings)

200000

In [11]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [41]:
ratingsValid[:10]

[('u59070515', 'b55084829', 5),
 ('u05014036', 'b49553867', 4),
 ('u97467443', 'b60783224', 5),
 ('u89187024', 'b67956548', 1),
 ('u41880097', 'b29998162', 3),
 ('u43993833', 'b10136233', 5),
 ('u76634269', 'b37466269', 2),
 ('u58694848', 'b24500781', 5),
 ('u00634922', 'b74770858', 3),
 ('u45282198', 'b23760358', 4)]

In [12]:
##################################################
# Rating prediction                              #
##################################################

In [17]:
trainRatings = [r[2] for r in ratingsTrain]

globalAverage = homework3.getGlobalAverage(trainRatings)
print(globalAverage)

3.6868052631578947


<module 'homework3' from 'c:\\Users\\Patric\\school\\FA25\\158\\158_hw\\assignment1\\homework3.py'>

In [21]:
def testQ1():
    ga = homework3.getGlobalAverage(trainRatings)
    trivialValidMSE = homework3.trivialValidMSE(ratingsValid, ga)
    
    print("average = " + str(ga))
    print("validation MSE = " + str(trivialValidMSE))

In [22]:
testQ1()

average = 3.6868052631578947
validation MSE = 1.6802113179224376


In [23]:
def iterateN(which, alpha, betaU, betaI, lamb, N):
    for i in range(N):
        alpha = which.alphaUpdate(ratingsTrain, alpha, betaU, betaI, lamb)
        betaU = which.betaUUpdate(ratingsPerUser, alpha, betaU, betaI, lamb)
        betaI = which.betaIUpdate(ratingsPerItem, alpha, betaU, betaI, lamb)
        mse, mseReg = which.msePlusReg(ratingsTrain, alpha, betaU, betaI, lamb)
        print("Iteration " + str(i + 1))
        print("  MSE = " + str(mse))
        print("  regularized objective = " + str(mseReg))
    return alpha, betaU, betaI, mse, mseReg

In [24]:
def testModel(which):
    betaU = {}
    betaI = {}
    for u in ratingsPerUser:
        betaU[u] = 0

    for b in ratingsPerItem:
        betaI[b] = 0

    alpha = globalAverage # Could initialize anywhere, this is a guess
    
    alpha, betaU, betaI, mse, mseReg = iterateN(which, alpha, betaU, betaI, 1.0, 1)
    validMSE = which.validMSE(ratingsValid, alpha, betaU, betaI)
    
    return alpha, betaU, betaI, mse, mseReg, validMSE

In [30]:
def testQ2():
    alpha, betaU, betaI, mse, mseReg, validMSE = testModel(homework3)
    print("validMSE = " + str(validMSE))

In [35]:
testQ2()

Iteration 1
  MSE = 1.072715470448089
  regularized objective = 13249.828429085357
validMSE = 1.440670105511255


In [36]:
def testQ3():
    betaU = {}
    betaI = {}
    for u in ratingsPerUser:
        betaU[u] = 0

    for b in ratingsPerItem:
        betaI[b] = 0

    alpha = globalAverage # Could initialize anywhere, this is a guess
    
    alpha, betaU, betaI = homework3.goodModel(ratingsTrain, ratingsPerUser, ratingsPerItem, alpha, betaU, betaI)
    validMSE = homework3.validMSE(ratingsValid, alpha, betaU, betaI)
    
    print("validMSE = " + str(validMSE))

In [ ]:
testQ3()

In [ ]:
##################################################
# Read prediction                                #
##################################################

In [39]:
# From baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [44]:
ratingsValid[:5]

[('u59070515', 'b55084829', 5),
 ('u05014036', 'b49553867', 4),
 ('u97467443', 'b60783224', 5),
 ('u89187024', 'b67956548', 1),
 ('u41880097', 'b29998162', 3)]

In [45]:
allRatings[:5]

[('u67805239', 'b61372131', 4),
 ('u54531895', 'b75189008', 4),
 ('u76549666', 'b75446982', 4),
 ('u03186275', 'b23482469', 2),
 ('u21322233', 'b09979253', 3)]

In [84]:
def testQ4():
    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)
    print("Should be equal: " + str((len(readValid), len(notRead), len(ratingsValid))))

In [85]:
testQ4()

Should be equal: (10000, 10000, 10000)


In [55]:
def testQ5():
    return1 = homework3.baseLineStrategy(mostPopular, totalRead)
    better = homework3.improvedStrategy(mostPopular, totalRead)
    
    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)
    
    correctA = homework3.evaluateStrategy(return1, readValid, notRead)
    correctB = homework3.evaluateStrategy(better, readValid, notRead)
    
    print("Accuracy (simple strategy) = " + str(correctA))
    print("Accuracy (better strategy) = " + str(correctB))

In [56]:
testQ5()

Accuracy (simple strategy) = 0.7092854642732137
Accuracy (better strategy) = 0.7354867743387169


In [59]:
def testQ6():
    readValid, notRead = homework3.generateValidation(allRatings, ratingsValid)
    
    for (u,b) in list(readValid)[:20] + list(notRead)[:20]:
        a = homework3.jaccardThresh(u,b,ratingsPerItem,ratingsPerUser)
        print("Jaccard-based predictor for " + str((u,b)) + " = " + str(a))

    # This is slow (so the autograder doesn't run it) but you should run it at home once you have a good solution
    homework3.writePredictionsRead(ratingsPerItem, ratingsPerUser)

In [60]:
testQ6()

Jaccard-based predictor for ('u26682101', 'b09399885') = 1
Jaccard-based predictor for ('u40124421', 'b93745301') = 1
Jaccard-based predictor for ('u79294617', 'b63603925') = 1
Jaccard-based predictor for ('u26550736', 'b95021405') = 1
Jaccard-based predictor for ('u33151348', 'b70562203') = 0
Jaccard-based predictor for ('u80899382', 'b40908409') = 1
Jaccard-based predictor for ('u54531895', 'b86859307') = 0
Jaccard-based predictor for ('u37429165', 'b32805710') = 0
Jaccard-based predictor for ('u54492852', 'b33470089') = 1
Jaccard-based predictor for ('u94996121', 'b52001713') = 0
Jaccard-based predictor for ('u17236627', 'b88182084') = 1
Jaccard-based predictor for ('u56553394', 'b36762401') = 0
Jaccard-based predictor for ('u02890572', 'b39738718') = 1
Jaccard-based predictor for ('u68377666', 'b17244552') = 1
Jaccard-based predictor for ('u37894318', 'b62853608') = 1
Jaccard-based predictor for ('u38239198', 'b72304066') = 1
Jaccard-based predictor for ('u35397545', 'b39047271') =

In [ ]:
##################################################
# Category prediction                            #
##################################################

In [61]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)
    # Just use a little data to make things faster...
    if len(data) > 10000:
        break

In [62]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [63]:
NW = 500 # dictionary size

In [64]:
words = [x[1] for x in counts[:NW]]

In [65]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [87]:
import importlib
importlib.reload(homework3)

<module 'homework3' from 'c:\\Users\\Patric\\school\\FA25\\158\\158_hw\\assignment1\\homework3.py'>

In [74]:
wordId

{'the': 0,
 'and': 1,
 'a': 2,
 'to': 3,
 'i': 4,
 'of': 5,
 'is': 6,
 'in': 7,
 'it': 8,
 'this': 9,
 'that': 10,
 'was': 11,
 'book': 12,
 'for': 13,
 'but': 14,
 'with': 15,
 'her': 16,
 'as': 17,
 'on': 18,
 'not': 19,
 'you': 20,
 'read': 21,
 'so': 22,
 'she': 23,
 'story': 24,
 'be': 25,
 'are': 26,
 'have': 27,
 'me': 28,
 'one': 29,
 'he': 30,
 'my': 31,
 'his': 32,
 'like': 33,
 'all': 34,
 'about': 35,
 'at': 36,
 'more': 37,
 'from': 38,
 'just': 39,
 'an': 40,
 'really': 41,
 'its': 42,
 'they': 43,
 'what': 44,
 'has': 45,
 'there': 46,
 'who': 47,
 'out': 48,
 'characters': 49,
 'by': 50,
 'up': 51,
 'when': 52,
 'series': 53,
 'or': 54,
 'love': 55,
 'some': 56,
 'if': 57,
 'will': 58,
 'had': 59,
 'very': 60,
 'first': 61,
 'would': 62,
 'how': 63,
 'good': 64,
 'their': 65,
 'were': 66,
 'much': 67,
 'because': 68,
 'books': 69,
 'time': 70,
 'into': 71,
 'them': 72,
 'reading': 73,
 'get': 74,
 'can': 75,
 'no': 76,
 'well': 77,
 'im': 78,
 'even': 79,
 'way': 80,
 '

In [82]:
def testQ7():
    f1 = homework3.featureCat(data[0], words, wordId, wordSet)
    
    print("Feature vector = " + str(f1))

In [83]:
testQ7()

Feature vector = [0, 2, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [88]:
def testQ8():
    X = [homework3.featureCat(d, words, wordId, wordSet) for d in data]
    y = [d['genreID'] for d in data]
    
    Xtrain = X[:9*len(X)//10]
    ytrain = y[:9*len(y)//10]
    Xvalid = X[9*len(X)//10:]
    yvalid = y[9*len(y)//10:]
    
    mod = linear_model.LogisticRegression(C=1)
    mod.fit(Xtrain, ytrain)
    pred = mod.predict(Xvalid)
    correctA = pred == yvalid
    correctA = sum(correctA) / len(correctA)
    
    X = homework3.betterFeatures(data)
    Xtrain = X[:9*len(X)//10]
    Xvalid = X[9*len(X)//10:]
    
    mod = linear_model.LogisticRegression(C=1)
    mod.fit(Xtrain, ytrain)
    pred = mod.predict(Xvalid)
    correctB = pred == yvalid
    correctB = sum(correctB) / len(correctB)
    
    sc = correctA < (correctB * 0.99)

    data_test = []
    for d in readGz("test_Category.json.gz"):
        data_test.append(d)
    
    Xtest = homework3.betterFeatures(data_test)
    pred_test = mod.predict(Xtest)
    
    homework3.writePredictionsCategory(pred_test)
    
    if sc:
        print("Looks like your solution is better")
    else:
        print("Looks like your solution is not better")

In [89]:
testQ8()

c:\Users\Patric\school\FA25\158\158_hw\assignment1\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Patric\school\FA25\158\158_hw\assignment1\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://sci

Looks like your solution is better
